In [1]:
import os 
import sys 

cur_path = os.path.abspath("../..")
if cur_path not in sys.path: 
    sys.path.append(cur_path)

from functools import cache 
import numpy as np 
import pandas as pd 
import altair as alt 
from IPython.display import clear_output
from altair import datum
from dotenv import load_dotenv
from subgrounds.subgrounds import Subgrounds, Subgraph
from subgrounds.pagination import ShallowStrategy

# Required when developing in a jupyter-notebook environment 
load_dotenv('../../../.env')

from utils_notebook.utils import ddf, load_subgraph, remove_prefix
from utils_notebook.vega import (
    output_chart, 
    apply_css, 
    stack_order_expr, 
    wide_to_longwide, 
    chart_stack_area_overlay_line_timeseries,
)
from utils_notebook.css import css_tooltip_timeseries_multi_colored

In [2]:
sg: Subgrounds
bs: Subgraph
sg, bs = load_subgraph()

In [3]:
def query_farmers_market_snaps(refresh=None):    
    snaps = bs.Query.podMarketplaceDailySnapshots(first=10000, orderBy="season", orderDirection='asc')
    bs.PodMarketplaceDailySnapshot.total_bean_vol = bs.PodMarketplaceDailySnapshot.totalBeanVolume / 1e6
    bs.PodMarketplaceDailySnapshot.total_pod_vol = bs.PodMarketplaceDailySnapshot.totalPodVolume / 1e6
    bs.PodMarketplaceDailySnapshot.total_pod_listing_vol = bs.PodMarketplaceDailySnapshot.totalPodsFilled / 1e6
    bs.PodMarketplaceDailySnapshot.total_pod_order_vol = bs.PodMarketplaceDailySnapshot.totalOrdersFilled / 1e6
    df = sg.query_df([
        snaps.timestamp, 
        snaps.season, 
        snaps.total_bean_vol, 
        snaps.total_pod_vol,
        snaps.total_pod_listing_vol, 
        snaps.total_pod_order_vol, 
    ], pagination_strategy=ShallowStrategy)
    df = remove_prefix(df, 'podMarketplaceDailySnapshots_')
    return df 

In [4]:
df_snaps = query_farmers_market_snaps()
col_map = {
    'total_bean_vol': 'total bean vol', 
    'total_pod_listing_vol': 'pod listing vol', 
    'total_pod_order_vol': 'pod order vol', 
    'total_pod_vol': 'total pod vol',
}
df_snaps['timestamp'] = pd.to_datetime(df_snaps.timestamp, unit="s")
df_snaps = df_snaps.rename(columns=col_map)
df_snaps = df_snaps.loc[
    (df_snaps['total bean vol'] != 0) & 
    (df_snaps['total pod vol'] != 0) & 
    (df_snaps['pod listing vol'] != 0) & 
    (df_snaps['pod order vol'] != 0)
].reset_index(drop=True)

In [5]:
id_vars = ['timestamp', 'season']
value_vars = [
    'pod listing vol',
    'pod order vol',
    'total bean vol',
    'total pod vol',
]
df_snaps = wide_to_longwide(df_snaps, "timestamp", id_vars, value_vars, sort_col="timestamp") 
df_snaps.head()

,timestamp,season,variable,value,pod listing vol,pod order vol,total bean vol,total pod vol
0,2022-02-06 22:31:18,4405,pod listing vol,168079.947589,168079.947589,69477.354604,37016.310151,237557.302193
1,2022-02-06 22:31:18,4405,pod order vol,69477.354604,168079.947589,69477.354604,37016.310151,237557.302193
2,2022-02-06 22:31:18,4405,total bean vol,37016.310151,168079.947589,69477.354604,37016.310151,237557.302193
3,2022-02-06 22:31:18,4405,total pod vol,237557.302193,168079.947589,69477.354604,37016.310151,237557.302193
4,2022-02-07 23:47:49,4429,pod listing vol,277435.017194,277435.017194,229091.907755,119166.663005,506526.924949


In [6]:
colors = {
    'pod listing vol': '#B5E48C', # light green yellow 
    'pod order vol': '#52B69A', # light blue green 
    'total bean vol': '#168AAD', # darker blue
    'total pod vol': '#184E77', # mid blue 
}
c = chart_stack_area_overlay_line_timeseries(
    df_snaps, 
    "timestamp", 
    value_vars, 
    ['pod listing vol', 'pod order vol', 'total bean vol',], 
    "Farmer's Market Volume", 
    yaxis_area_kwargs=dict(title="Volume", format=".3~s"), 
    color_map=colors, 
)

css_lines = css_tooltip_timeseries_multi_colored(value_vars, colors) 
css = "\n".join(css_lines)

apply_css("")
# apply_css(css)

c

alt.LayerChart(...)

In [7]:
output_chart(c, css=css)

<IPython.core.display.JSON object>